<a href="https://colab.research.google.com/github/Fahim2q/Alumni_project/blob/main/Strawberry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#1
!pip install tensorflow


In [2]:
#2
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.keras import models,layers
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt


Mounted at /content/drive


In [3]:
#2.5 = Adding mixed precision for better gpu utilization
from tensorflow.keras import mixed_precision

# Enable mixed precision globally (GPU-accelerated)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

print("✅ Mixed precision enabled:", mixed_precision.global_policy())


✅ Mixed precision enabled: <DTypePolicy "mixed_float16">


In [4]:
#3
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3
EPOCHS = 200


In [5]:
#4
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Colab Notebooks/strawberry",  # Path to your dataset
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)


NotFoundError: Could not find directory /content/drive/MyDrive/Colab Notebooks/strawberry

In [ ]:
#5
class_names=dataset.class_names
class_names


In [ ]:
#6
for image_batch,layers_batch in dataset.take(1):
    print(image_batch.shape)
    print(layers_batch.numpy())

   # for i in range(min(12, len(image_batch))):  # Adjust loop to fit batch size
    for i in range(12):
        ax=plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[layers_batch[i]])
        plt.axis("off")


In [ ]:

#7
len(dataset)


In [ ]:

#7
len(dataset)


In [ ]:

#9
def get_dataset_partition_tf(ds, train_spilt=0.6,val_spilt=0.2,test_spilt=0.2,shuffle=True,shuffle_size=200):
    assert(train_spilt+val_spilt+test_spilt) == 1
    ds_size=len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=12)
    train_size=int(ds_size*train_spilt)
    val_size=int(ds_size*val_spilt)

    train_ds=ds.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)

    return train_ds,val_ds,test_ds


In [ ]:
#10
train_ds,val_ds,test_ds=get_dataset_partition_tf(dataset)


In [ ]:
#10.5 - Automatic Class Balancing
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter

print("📊 CALCULATING CLASS DISTRIBUTION FOR AUTOMATIC BALANCING...")

# Extract all labels from training dataset
train_labels = []
for _, labels_batch in train_ds:
    train_labels.extend(labels_batch.numpy())

# Count samples per class
class_counts = Counter(train_labels)
total_samples = len(train_labels)

# Display class distribution
print(f"\n📈 CLASS DISTRIBUTION (Total: {total_samples} samples):")
for class_idx, count in sorted(class_counts.items()):
    percentage = (count / total_samples) * 100
    print(f"  {class_names[class_idx]:15} → {count:4d} samples ({percentage:.1f}%)")

# Calculate class weights using sklearn's balanced approach
unique_classes = np.unique(train_labels)
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=unique_classes,
    y=train_labels
)
# Convert to dictionary format for Keras
class_weights = {i: weight for i, weight in enumerate(class_weights_array)}

print(f"\n⚖️  CALCULATED CLASS WEIGHTS:")
for class_idx, weight in class_weights.items():
    print(f"  {class_names[class_idx]:15} → {weight:.3f}")

print(f"\n✅ Class balancing ready! Higher weights assigned to minority classes.")




In [ ]:

#11
len(train_ds)


In [ ]:
#12
len(val_ds)


In [ ]:

#13
len(test_ds)


In [ ]:
#14
(47*32)+(15*32)+(17*32)


In [ ]:
#15 (smart caching with auto-clean for disk cache)
# (also with summary table)
import psutil, os, shutil, time

cache_summary = {}

def optimize_dataset(ds, name="", shuffle_buffer=1000):
    # Estimate available memory (in GB)
    available_gb = psutil.virtual_memory().available / (1024**3)

    # Estimate dataset size in GB (approx for float32)
    est_size_gb = (len(ds) * BATCH_SIZE * 0.00075)

    # Check /tmp free space
    tmp_usage = psutil.disk_usage("/tmp")
    tmp_free_gb = tmp_usage.free / (1024**3)

    print(f"\n📂 {name.upper()} dataset estimated size: {est_size_gb:.2f} GB")
    print(f"💻 Available RAM: {available_gb:.2f} GB | 💾 Free /tmp space: {tmp_free_gb:.2f} GB")

    if est_size_gb < available_gb * 0.3:
        # Safe to cache in RAM
        print(f"✅ Using RAM cache() for {name} dataset")
        ds = ds.cache().shuffle(shuffle_buffer).prefetch(buffer_size=tf.data.AUTOTUNE)
        cache_summary[name] = "RAM cache"

    else:
        # Too large for RAM → cache to disk if space allows
        cache_path = f"/tmp/{name}_cache"

        if est_size_gb > tmp_free_gb:
            print(f"❌ WARNING: {name} dataset (~{est_size_gb:.2f} GB) > free /tmp ({tmp_free_gb:.2f} GB). Skipping cache.")
            ds = ds.shuffle(shuffle_buffer).prefetch(buffer_size=tf.data.AUTOTUNE)
            cache_summary[name] = "No cache"
            return ds

        # Auto-clean old cache
        if os.path.exists(cache_path):
            print(f"🧹 Removing old cache at {cache_path}")
            shutil.rmtree(cache_path)

        print(f"💾 Caching {name} dataset to disk at {cache_path}...")
        start = time.time()
        ds = ds.cache(cache_path).shuffle(shuffle_buffer).prefetch(buffer_size=tf.data.AUTOTUNE)
        print(f"✅ Finished caching {name} dataset in {time.time()-start:.2f} sec")
        cache_summary[name] = "Disk cache"

    return ds


# Apply optimization
train_ds = optimize_dataset(train_ds, "train")
val_ds   = optimize_dataset(val_ds, "val")
test_ds  = optimize_dataset(test_ds, "test")

# Print final summary
print("\n📊 DATASET CACHING SUMMARY:")
for k, v in cache_summary.items():
    print(f" - {k.upper()} → {v}")



#16
for image_batch, layers_batch in dataset.take(1):
    print(image_batch[0].numpy()/255)


#17
resizing_and_rescaling=tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])



#18
data_augmentation=tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])



In [ ]:
#19 - Original dataset augmentation for training
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:

#20 - Optimized model (augmentation stays inside, no redundancy)
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = len(class_names)

model = models.Sequential([
    resizing_and_rescaling,
    data_augmentation,   # augmentation happens here only (GPU-accelerated)

    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),

 #layers.Flatten(),
    layers.GlobalAveragePooling2D(), #flatten replaced with GAP
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(n_classes, activation='softmax')
])


In [ ]:
#20.5
# Epoch Time Callback
import time

class EpochTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.epoch_times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        duration = time.time() - self.start_time
        self.epoch_times.append(duration)
        print(f"⏱️ Epoch {epoch+1} took {duration:.2f} sec")

epoch_timer = EpochTimeCallback()


In [ ]:

#21
model.summary()


In [ ]:
#22
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

checkpoint = ModelCheckpoint(
    "/content/drive/MyDrive/lumpy_diseases_detection/best_model.h5",
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)


In [ ]:
#23
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])


In [ ]:

#24 - Robust Training with Checkpoint and Epoch Saver
import os
import tensorflow as tf
import numpy as np

# Paths for checkpoint and epoch saving
checkpoint_path = "/content/drive/MyDrive/lumpy_diseases_detection/last_model.h5"
epoch_file = "/content/drive/MyDrive/lumpy_diseases_detection/last_epoch.npy"

# Define the EpochSaver callback
class EpochSaver(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        np.save(epoch_file, epoch)

epoch_saver = EpochSaver()

# Callbacks for early stopping and saving best model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint_cb = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Initialize starting epoch
initial_epoch = 0

# Check if checkpoint exists
if os.path.exists(checkpoint_path):
    print("✅ Resuming training from checkpoint...")
    model = tf.keras.models.load_model(checkpoint_path)

    # Recompile after loading
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

    # Load last saved epoch if available
    if os.path.exists(epoch_file):
        initial_epoch = int(np.load(epoch_file)) + 1
        print(f"Resuming from epoch {initial_epoch}")
else:
    print("⚡ No checkpoint found. Starting training from scratch.")
    # Use model already defined in #20
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

# Start training
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    initial_epoch=initial_epoch,
    class_weight=class_weights,  # ← ADD  (for class balancing)
    callbacks=[early_stop, checkpoint_cb, epoch_saver, epoch_timer]
)


In [ ]:
#25
len(test_ds)


In [ ]:
#26
scores=model.evaluate(test_ds)


In [ ]:
#27
history.history.keys()


In [ ]:
#28
history.history['loss']


In [ ]:
#29
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']


In [ ]:
#30
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(len(acc)), acc, label='Training_accuracy')
plt.plot(range(len(val_acc)), val_acc, label='Validation_accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(range(len(loss)), loss, label='Training_loss')
plt.plot(range(len(val_loss)), val_loss, label='Validation_loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:

#31
import numpy as np
actual=[]
predicted=[]
for images_batch, labels_batch in test_ds.take(24):  # 17 batches
    for i in range(len(images_batch)):  # handle last batch < 32 safely
        image = images_batch[i].numpy().astype("uint8")
        label = labels_batch[i].numpy()

        print(f"\nImage {i}")
        print("Actual Disease: ", class_names[label])
        actual.append(class_names[label])

        # Add batch dimension to the image
        prediction = model.predict(tf.expand_dims(images_batch[i], axis=0))
        predicted_class = class_names[np.argmax(prediction[0])]

        print("Predicted Disease: ", predicted_class)
        predicted.append(predicted_class)


In [ ]:

#32
print(classification_report(actual, predicted))


In [ ]:
#33
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Generate the confusion matrix
cm = confusion_matrix(actual, predicted, labels=class_names)

# Plot it
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
#34
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)  # fixed
    img_array = tf.expand_dims(img_array, 0)

    prediction = model.predict(img_array)
    predict_cls = class_names[np.argmax(prediction[0])]
    confidence = round(100 * (np.max(prediction[0])), 2)
    return predict_cls, confidence


In [ ]:
#35
plt.figure(figsize=(15,15))
for images,labels in test_ds.take(1):
    for i in range(9):
        ax=plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence= predict(model,images[i].numpy())

        actual_cls=class_names[labels[i]]
        plt.title(f"Actual: {actual_cls} \n Predicted : {predicted_class} \n Confidence : {confidence}%")
        plt.axis("off")


In [ ]:
#36
model.save("/content/drive/MyDrive/lumpy_diseases_detection/cnn_lumpy_detection.h5")


In [ ]:
#37
import tensorflow as tf

# Load the saved model
model = tf.keras.models.load_model("/content/drive/MyDrive/lumpy_diseases_detection/cnn_lumpy_detection.h5")

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# (Optional) Enable optimizations for smaller size and better performance
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the converted model to a file
with open("/content/drive/MyDrive/lumpy_diseases_detection/cnn_lumpy_detection_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model has been successfully created!")

